In [1]:
import os 
import pandas as pd 
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
from datetime import timedelta
import re
import paramiko
import numpy as np
import shutil
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot


pd.options.plotting.backend = "plotly"

In [54]:
def find_dir_without_graph(RESU_dir):
    date = []
    filenames = []
    dico = {}
    for filename in os.scandir(RESU_dir):
        if filename.is_dir():
            subfolder = RESU_dir.joinpath(filename.name)
            is_graph = False
            run_solver = False
            error = False
            for entry in os.scandir(subfolder):
                if entry.is_dir():
                    if re.search("Graph",entry.name):
                        is_graph = True
                if re.search("run_solver.log",entry.name):
                    run_solver = True
                if re.search("error",entry.name):
                    error = True
            if not is_graph and run_solver and not error:
                filenames.append(filename.name)
    return filenames


def preprocess_listing(res_dir,output_name='output.txt'):
    try: 
        os.mkdir(res_dir.joinpath('Graphs')) 
    except OSError as error: 
        print('Folder exists, continue')
    f = open(res_dir.joinpath("run_solver.log"),'r')
    res_dir = res_dir.joinpath('Graphs')  
    f2 = open(res_dir.joinpath(output_name),'w')
    lines = f.readlines()
    start = False
    for l in lines:
        l=l.replace("[","")
        l=l.replace("]","")
        a = re.search("v  |c  |INSTANT",l)
        if not start:
            if re.search('INSTANT',l):
                start = True
        if a and start:
            f2.write(l)
    f.close()
    f2.close()
    return res_dir.joinpath(output_name)

def extract_variable(var,filepath):
    f3 = open(filepath,'r')
    lines = f3.readlines()
    ts , content = [],[]
    for l in lines:
        a = re.search("INSTANT.*NUMBER *([0-9]*)",l)
        if a:
            ts.append(int(a.group(1)))
        b = re.search(var+' *([0-9\.eE\+-]*) *([0-9\.eE\+-]*) *([0-9\.eE\+-]*) *([0-9\.eE\+-]*) *([0-9\.eE\+-]*)',l)
        if b:
            data = []
            for g in b.groups():
                if g != '':
                    data.append(float(g))
            content.append(data)
    a = min(len(ts),len(content))
    if (len(ts) != len(content)):
        print("ATTENTION : Troncature extract variable")
        print("Fichier : ",filepath)
        print("Nb ts : ",len(ts), " Nb pas extraits ",len(content))
    return ts[:a],content[:a]

def extract_fields(filepath, variables, fields):
    '''
    Extract the fields value of the specified variables
    Variables are expected to be formated like this : 
    c  var_name
    v  var_name 
    '''
    df_exist = False
    for i in range(len(variables)):
        variable = variables[i]
        col_names = []
        res = re.search('(c|v)  (\S*)',variable)
        if res:
            var_name = res.group(2)
            for var in fields:
                col_names.append((var_name,var))
            ts,content = extract_variable(variable,filepath)
            if (len(ts)>1):
                if not df_exist:
                    df = pd.DataFrame(content,index=pd.Index(ts,name="Timestep"))
                    df.columns = pd.MultiIndex.from_tuples(col_names)
                    df_exist = True
                else:
                    df1 = pd.DataFrame(content,index=pd.Index(ts,name="Timestep"))
                    df1.columns = pd.MultiIndex.from_tuples(col_names)
                    df = df.join(df1)
    if df_exist:
        df = df.swaplevel(axis=1)
        df = df.sort_index(axis=1)
        return df
    else:
        return None

def process_listing(RESU_dir,dir_list=None):
    temporary_file = 'output.txt'
    if dir_list == None:
        dir_list = find_dir_without_graph(RESU_dir)
    print(dir_list)
    for res in dir_list:
        result = RESU_dir.joinpath(res)
        output_path = preprocess_listing(result,temporary_file)
        resu_path = output_path.parents[0]
        value_fields = ['minimum','maximum','set_mean','spatial_mean']
        value_var = ['v  ǁVelocityǁ','v  Pressure','v  k','v  epsilon','v  CourantNb','v  VelocityX','v  VelocityY']
        conv_fields = ['Rhs norm','N_iter','Norm. residual','Drift','Time residual']
        conv_var = ['c  Velocity','c  Pressure','c  k','c  epsilon']
        df_v = extract_fields(output_path, value_var, value_fields)
        df_c = extract_fields(output_path, conv_var, conv_fields)
        with open(resu_path.joinpath('convergence_graph.html'), 'w') as f:
            k_e_path = resu_path.parents[0].joinpath('KineticEnergy.csv')
            if os.path.isfile(k_e_path):
                df = pd.read_csv(k_e_path)
                df.index = df['timestep']
                df.drop(columns='timestep',inplace=True)
                fig = df.plot(labels={"value": "kinetic energy"})
                fig.update_layout(
                    title={'text':'Total Kinetic Energy', 'font':dict(size=30), 'y':0.95,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'},
                    updatemenus=[
                dict(buttons=[
                        dict(label="Linear",  
                            method="relayout", 
                            args=[{"yaxis.type": "linear"}]),
                        dict(label="Log", 
                            method="relayout", 
                            args=[{"yaxis.type": "log"}]),
                                    ])]
                
                )
                f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
            else:
                print('KineticEnergy.csv not found')
            for field in conv_fields:
                source_df = df_c[field].copy(deep=True)[10:]
                fig = source_df.plot(labels={"value": field+" value"})
                fig.update_layout(
                    title={'text':field, 'font':dict(size=30), 'y':0.95,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'}
                ,
                updatemenus=[
                dict(buttons=[
                     dict(label="Linear",  
                          method="relayout", 
                          args=[{"yaxis.type": "linear"}]),
                     dict(label="Log", 
                          method="relayout", 
                          args=[{"yaxis.type": "log"}]),
                                  ])]
            
                )
                f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

        with open(resu_path.joinpath('value_graph.html'), 'w') as f:
            for field in value_fields:
                source_df = df_v[field].copy(deep=True)[10:]
                fig = source_df.plot(labels={"value": field+" value"})
                fig.update_layout(
                    title={'text':field, 'font':dict(size=30), 'y':0.95,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'},
                    updatemenus=[
                    dict(buttons=[
                     dict(label="Linear",  
                          method="relayout", 
                          args=[{"yaxis.type": "linear"}]),
                     dict(label="Log", 
                          method="relayout", 
                          args=[{"yaxis.type": "log"}]),
                                  ])]
                )
                f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

In [55]:
Doc_dir = Path("/home/juleslecoustre/Documents/Acqua/PeriodicHill")
#Study_name = "FuelRod"
#Case_Name = "CASE1"

#Case_dir = Doc_dir.joinpath(Study_name).joinpath(Case_Name)
#Res_dir = Case_dir.joinpath("RESU"),dir_list=["LRE_1mm_22June7dbg"]
process_listing(Doc_dir)

['Cubic_periodic_BC_2ndTS', 'Cubic_periodic_BC', 'STRUCT-Epsilon', 'SKE_periodic_BC']
ATTENTION : Troncature extract variable
Fichier :  /home/juleslecoustre/Documents/Acqua/PeriodicHill/Cubic_periodic_BC_2ndTS/Graphs/output.txt
Nb ts :  160  Nb pas extraits  0
ATTENTION : Troncature extract variable
Fichier :  /home/juleslecoustre/Documents/Acqua/PeriodicHill/Cubic_periodic_BC_2ndTS/Graphs/output.txt
Nb ts :  160  Nb pas extraits  0
ATTENTION : Troncature extract variable
Fichier :  /home/juleslecoustre/Documents/Acqua/PeriodicHill/Cubic_periodic_BC_2ndTS/Graphs/output.txt
Nb ts :  160  Nb pas extraits  0
ATTENTION : Troncature extract variable
Fichier :  /home/juleslecoustre/Documents/Acqua/PeriodicHill/Cubic_periodic_BC_2ndTS/Graphs/output.txt
Nb ts :  160  Nb pas extraits  0
ATTENTION : Troncature extract variable
Fichier :  /home/juleslecoustre/Documents/Acqua/PeriodicHill/STRUCT-Epsilon/Graphs/output.txt
Nb ts :  210  Nb pas extraits  0
ATTENTION : Troncature extract variable
Fic